In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from unidecode import unidecode
import requests
from tqdm import tqdm

In [17]:
page_link = 'https://fbref.com/en/comps/season/2017-2018'
page = requests.get(page_link)
bs_page = BeautifulSoup(page.content,'html.parser')

In [18]:
top_5_tournaments = ['Premier League','Bundesliga','Ligue 1 Conforama','LaLiga Santander','Serie A TIM']

In [19]:
domestic_tournments_section = bs_page.find('div',id='all_comps_1_fa_club_league_senior')

In [20]:
tournament_table = domestic_tournments_section.find('div',class_='table_outer_container')

In [21]:
tournament_table_body = tournament_table.find('tbody')

In [22]:
tournament_names_links = dict()
for tourn in tournament_table_body.find_all('th',attrs={'data-stat':True}):
    results_page_links ='http://www.fbref.com'+'/'.join(tourn.find('a')['href'].split('/')[:-1])+'/schedule/' + '-'.join(tourn.find('a')['href'].split('/')[-1].split('-')[:-1])+'-Scores-and-Fixtures'
    tournament_names_links[tourn.find('a').get_text()] = results_page_links

In [23]:
tournament_names_links

{'Superliga Argentina': 'http://www.fbref.com/en/comps/21/1659/schedule/2017-2018-Superliga-Argentina-Scores-and-Fixtures',
 'Hyundai A-League': 'http://www.fbref.com/en/comps/65/1617/schedule/2017-2018-A-League-Scores-and-Fixtures',
 'tipico Bundesliga': 'http://www.fbref.com/en/comps/56/1618/schedule/2017-2018-Austrian-Bundesliga-Scores-and-Fixtures',
 'Jupiler Pro League': 'http://www.fbref.com/en/comps/37/1619/schedule/2017-2018-Belgian-First-Division-A-Scores-and-Fixtures',
 'First League': 'http://www.fbref.com/en/comps/67/1620/schedule/2017-2018-Bulgarian-First-League-Scores-and-Fixtures',
 'Hrvatski telekom Prva liga': 'http://www.fbref.com/en/comps/63/1622/schedule/2017-2018-1-HNL-Scores-and-Fixtures',
 'HET liga': 'http://www.fbref.com/en/comps/66/1623/schedule/2017-2018-Czech-First-League-Scores-and-Fixtures',
 'Alka Superliga': 'http://www.fbref.com/en/comps/50/1624/schedule/2017-2018-Superliga-Scores-and-Fixtures',
 'Premier League': 'http://www.fbref.com/en/comps/9/1631/s

In [24]:
matches_info = list()
for tournament in top_5_tournaments:
    print(tournament)
    results_link = tournament_names_links[tournament]
    tournament_results_page = requests.get(results_link)
    soup = BeautifulSoup(tournament_results_page.content,'html.parser')
    tournament_results_container = soup.find('div',class_='table_outer_container')
    tournament_results_body = tournament_results_container.find('tbody')
    for c in tournament_results_body.findChildren('tr',recursive=False):
        if len(c.attrs)==0:
            try:
                round_text = c.find('th',{'data-stat':'round'}).find('a').get_text()
                flag = 1
            except:
                flag = 0

            if flag == 1:
                if 'Regular' in round_text:
                    match_dict={}
                    match_dict['tournament'] = tournament
                    match_dict['gameweek'] = c.find('td',{'data-stat':'gameweek'}).get_text()
                    match_dict['date'] = c.find('td',{'data-stat':'date'}).find('a').get_text()
                    match_dict['home_team'] = c.find('td',{'data-stat':'squad_a'}).find('a').get_text()
                    match_dict['score'] = c.find('td',{'data-stat':'score'}).find('a').get_text()
                    match_dict['away_team'] = c.find('td',{'data-stat':'squad_b'}).find('a').get_text()
                    match_dict['match_link'] = 'http://www.fbref.com'+c.find('td',{'data-stat':'match_report'}).find('a')['href']
                    matches_info.append(match_dict)
            else:
                match_dict={}
                match_dict['tournament'] = tournament
                match_dict['gameweek'] = c.find('th',{'data-stat':'gameweek'}).get_text()
                match_dict['date'] = c.find('td',{'data-stat':'date'}).find('a').get_text()
                match_dict['home_team'] = c.find('td',{'data-stat':'squad_a'}).find('a').get_text()
                match_dict['score'] = c.find('td',{'data-stat':'score'}).find('a').get_text()
                match_dict['away_team'] = c.find('td',{'data-stat':'squad_b'}).find('a').get_text()
                match_dict['match_link'] = 'http://www.fbref.com'+c.find('td',{'data-stat':'match_report'}).find('a')['href']
                matches_info.append(match_dict)



    
    

Premier League
Bundesliga
Ligue 1 Conforama
LaLiga Santander
Serie A TIM


In [25]:
for match_info in tqdm(matches_info):
        match_page = requests.get(match_info['match_link'])
        match_soup = BeautifulSoup(match_page.content,'html.parser')
        field = match_soup.find('div',id='field')
        home_team_match_defs = dict()
        away_team_match_defs = dict()
        for player in field.find_all('div'):
            if player['class'][1]=='a' and 'calc(15% - 8px)' in  player['style'].split(';')[1].strip():
                player_name = player['title'].strip()
                player_name = re.sub('[^a-zA-Z]+','',unidecode(player_name))
                home_team_match_defs[player_name] = float(re.search('\((.+?)%',player['style'].split(';')[0].strip().split(':')[1].strip()).group(1))
            elif player['class'][1]=='b' and 'calc(15% - 8px)' in  player['style'].split(';')[1].strip():
                player_name = player['title'].strip()
                player_name = re.sub('[^a-zA-Z]+','',unidecode(player_name))
                away_team_match_defs[player_name] = float(re.search('\((.+?)%',player['style'].split(';')[0].strip().split(':')[1].strip()).group(1))  
        match_info['home_team_defs'] = sorted(home_team_match_defs,key=home_team_match_defs.get,reverse=True)
        match_info['away_team_defs'] = sorted(away_team_match_defs,key=away_team_match_defs.get)

100%|██████████| 1826/1826 [1:18:27<00:00,  2.58s/it]


In [30]:
df_matches_info = pd.DataFrame(matches_info)

In [31]:
df_matches_info.to_pickle('../data_top5/matches/top_5_lineups.pkl')